# [Day 20](https://adventofcode.com/2020/day/20): Jurassic Jigsaw

In [1]:
import numpy as np
import re

with open("../data/20.txt", "r") as f:
    lines = "".join(l.strip() for l in f.readlines())    
    id_tile = re.findall(r"Tile (\d+):([\.|#]+)", lines)

ids = np.array([int(i) for i, _ in id_tile])
tiles = np.array([list(t) for _, t in id_tile]) == "#"
tiles = tiles.reshape(-1, 10, 10)

## Part 1

In [2]:
from itertools import chain

def border(tile):
    es = tile[0, :], tile[:, -1], tile[-1, :], tile[:, 0]
    return map(tuple, chain(es, map(reversed, es)))

In [3]:
from collections import defaultdict, Counter

def boundary(tiles):
    t = defaultdict(set)
    for i, es in enumerate(map(border, tiles)):
        for e in es:
            t[e].add(i)
    cts = Counter(i.pop() for _, i in t.items() if len(i) == 1)
    sides = [i for i, n in cts.items() if n == 2]
    corners = [i for i, n in cts.items() if n == 4]
    edges = np.array([e for e, i in t.items() if not i])
    return edges, sides, corners

In [4]:
edges, sides, corners = boundary(tiles)
assert 21599955909991 == np.prod(ids[corners])

## Part 2

In [5]:
from functools import reduce

def rings(tiles):
    tiles = tiles.copy()
    nw = None
    while len(tiles) > 0:
        edges, sides, corners = boundary(tiles)
        rings = list(arrange(edges, tiles[sides], tiles[corners], nw))
        yield from rings
        nw = rings[1][-1], rings[-1][:, -1]
        tiles = inner(tiles, sides, corners)

def inner(tiles, sides, corners):
    inside = np.ones(len(tiles), dtype=bool)
    inside[sides] = False
    inside[corners] = False
    return tiles[inside]

def arrange(edges, sides, corners, nw):
    sides, corners = normalize(edges, sides, corners)
    n = len(sides) // 4
    if nw is None:
        corner = corners[0]
        corners = corners[1:]
    else:
        for i, c in enumerate(corners):
            if np.all(c[0] == nw[0]) and np.all(c[:, 0] == nw[1]):
                break
            c = c.T
            if np.all(c[0] == nw[0]) and np.all(c[:, 0] == nw[1]):
                break
        corner = c
        corners = corners[np.arange(4) != i]
    for k in range(4):
        edge = corner[:, -1]
        yield turn_clockwise(corner, k)
        for _ in range(n):
            for i, s in enumerate(sides):
                l, r = s[:, 0], s[:, -1]
                if np.all(l == edge):
                    edge = r
                    yield turn_clockwise(s, k)
                    break
                if np.all(r == edge):
                    edge = l
                    yield turn_clockwise(s[:, ::-1], k)
                    break
            sides = sides[np.arange(len(sides)) != i]
        for i, corner in enumerate(corners):
            if np.all(corner[-1] == edge):
                break
            if np.all(corner[:, -1] == edge):
                corner = corner.T
                break
        corners = corners[np.arange(len(corners)) != i]
    
def normalize(edges, sides, corners):
    edges = edges[np.newaxis, np.newaxis, :, :]
    if len(sides):
        sides = list(map(turn_counterclockwise, sides, np.argmax(isouter(edges, sides), axis=1)))
    else:
        sides = []
    corners = list(map(turn_counterclockwise, corners, map(nturns, isouter(edges, corners))))
    return np.array(sides), np.array(corners)

def isouter(edges, tiles):
    o = np.array(list(map(bd, tiles)))[:, :, np.newaxis, :] == edges
    return np.any(np.all(o, axis=3), axis=2)

def bd(tile):
    return tile[0, :], tile[:, -1], tile[-1, :], tile[:, 0]

def turn_counterclockwise(tile, n):
    return reduce(lambda t, _: t.T[::-1], range(n), tile)

def turn_clockwise(tile, n):
    return reduce(lambda t, _: t.T[:, ::-1], range(n), tile)

def nturns(isouter, NWCORNER=np.array([True, False, False, True])):
    return next(n for n in range(4) if np.all(np.roll(isouter, -n) == NWCORNER))

In [6]:
def boundary_indices(n):
    i = np.zeros(4*(n + 1) - 8, dtype=int)
    i[n:(2*n - 1)] = np.arange(1, n)
    i[(2*n - 1):-n + 1] = n - 1
    i[-n + 1:] = np.arange(n - 1, 0, -1)
    return np.array([i, np.roll(i, -n + 1)]).T

In [7]:
tiles2 = np.array(list(rings(tiles)))[:, 1:-1, 1:-1]
grid = np.empty((96, 96), dtype=bool)
l = 0
for n, s in zip(range(12, 0, -2), range(0, 48, 8)):
    ij = boundary_indices(n)
    ring = tiles2[l:l+len(ij)]
    l += len(ij)
    for (i, j), r in zip(ij, ring):
        grid[(s+8*i):(s+8*(i+1)), (s+8*j):(s+8*(j+1))] = r

In [8]:
tiles3 = np.array(list(rings(tiles)))
grid3 = np.empty((120, 120), dtype=bool)
l = 0
for n, s in zip(range(12, 0, -2), range(0, 60, 10)):
    ij = boundary_indices(n)
    ring = tiles3[l:l+len(ij)]
    l += len(ij)
    for (i, j), r in zip(ij, ring):
        grid3[(s+10*i):(s+10*(i+1)), (s+10*j):(s+10*(j+1))] = r

In [9]:
for i in range(10, 120, 10):
    assert np.all(grid3[:, i-1] == grid3[:, i])
    assert np.all(grid3[i-1] == grid3[i])

In [10]:
monster = """\
                  # \
#    ##    ##    ###\
 #  #  #  #  #  #   \
"""
monster = np.array(list(monster)).reshape(3, -1)
monster = monster == "#"
monster = monster[np.newaxis, :, :]

In [11]:
from itertools import product

pos = list(product(range(96-3+1), range(96-20+1)))

def overlap_monster(grid):
    return monster & np.array([grid[i:i+3, j:j+20] for i, j in pos])

In [12]:
g = grid.copy()
for _ in range(4):
    overlaps = overlap_monster(g)
    n_monster = np.sum(np.sum(overlaps, axis=(1, 2)) == 15)
    print(n_monster, np.sum(g) - n_monster * 15)
    g = g.T[::-1]

0 2735
0 2735
0 2735
0 2735


In [13]:
g = grid[::-1].copy()
for _ in range(4):
    overlaps = overlap_monster(g)
    n_monster = np.sum(np.sum(overlaps, axis=(1, 2)) == 15)
    print(n_monster, np.sum(g) - n_monster * 15)
    g = g.T[::-1]

0 2735
0 2735
16 2495
0 2735


Answer for part two is `2495`.